# Topic Modeling: Beyond LDA

While LDA topic modeling's ability to pick up on latent themes in large collections of texts can be quite useful (hence the model's popularity), LDA models nevertheless have a number of limitations. To name a few, LDA models don't account for the passage of time, the models have difficulty determining any relationships among generated topics, and topics become considerably less useful when the model is applied to shorter corpera with shorter document lengths.

In this chapter, we'll present some alternative approaches to topic modeling that help to mitigate these limitations of LDA modeling.  

## Dynamic Topic Modeling 


In developing Dynamic Topic Modeling, or DTM, [Blei and Lafferty](https://dl.acm.org/doi/pdf/10.1145/1143844.1143859) wanted to account for the possibility that content within a collection of texts could evolve over time, something traditional LDA topic modeling doesn't consider. To do so, they developed a form of topic modeling that could trace the evolution of the topics generated over time. In DTM, then, we can see our topics develop as time passes.

### Gensim

In order to run Dynamic Topic Modeling in Python, we'll be installing the [Gensim](https://radimrehurek.com/gensim/index.html) topic modeling library. (We'll also want to make sure we've installed Gensim's dependencies.)

In [1]:
pip install gensim

From Gensim, we'll import [ldaseq](https://radimrehurek.com/gensim/models/ldaseqmodel.html), the library's built-in Dynamic Topic Modeling function.  

In [2]:
from gensim.models import LdaSeqModel

#### LdaSeqModel Parameters: 

- **corpus**: The collection of document vectors we'll use to fit our LDA Sequence model.


- **id2word**: Allows us to map word IDs onto words, and helps determinine the size of our vocabulary.



- **time_slice**: The number of documents we'd like to include within each period of time we want our model to consider. 


- **num_topics**: The total number of topics we'd like our model to determine.


- **passes**: This parameter functions in a similar manner to the `max_iter` parameter we set when running a conventional LDA model with scikit-learn. Like `max_iter`, `passes` is set to 10 by default. As was the case before, we'll almost always want to set `passes` to a number higher than 10.  


- **em_max_iter**: Sets a maximum threshold on the number of iterations until we reach convergence of the  [Expectation-Maximization](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) algorithm.

(*note to self: need a better link than Wikipedia*)



Just to get a handle on the `LdaSeqModel` function, we'll run the model with gensim's common corpus and dictionary. We'll use the common dictionary for the `id2word` parameter when we run our model.

In [3]:
from gensim.test.utils import common_corpus
from gensim.test.utils import common_dictionary

corpus = common_corpus
dictionary = common_dictionary

In [4]:
ldaseq = LdaSeqModel(corpus=common_corpus, id2word=dictionary, time_slice=[2, 4, 3], num_topics=2)

As you can see below, the topics generated with the Gensim common corpus and dictionary don't appear to vary much over time:

In [5]:
ldaseq.print_topics(time=0)

[[('system', 0.21157474442727292),
  ('user', 0.16464543889319655),
  ('computer', 0.11780520871853883),
  ('interface', 0.11780520871853883),
  ('eps', 0.06967385482884698),
  ('response', 0.06951097868862414),
  ('time', 0.06951097868862414),
  ('survey', 0.051180555909905195),
  ('trees', 0.032452567340626),
  ('graph', 0.032452567340626),
  ('minors', 0.032452567340626),
  ('human', 0.030935329104574284)],
 [('trees', 0.32492222725659636),
  ('graph', 0.3235563263440136),
  ('minors', 0.2205714063302323),
  ('computer', 0.0382488601600507),
  ('survey', 0.022503013800186142),
  ('human', 0.0100605551936047),
  ('interface', 0.0100605551936047),
  ('response', 0.0100605551936047),
  ('time', 0.0100605551936047),
  ('user', 0.0100605551936047),
  ('system', 0.00994769507044868),
  ('eps', 0.00994769507044868)]]

In [6]:
ldaseq.print_topics(time=1)

[[('system', 0.21240589151067885),
  ('user', 0.1646899189988433),
  ('computer', 0.11749357289440712),
  ('interface', 0.11749357289440712),
  ('eps', 0.06963604929122659),
  ('response', 0.0694731541329969),
  ('time', 0.0694731541329969),
  ('survey', 0.05114236405938337),
  ('trees', 0.03241859405373477),
  ('graph', 0.03241859405373477),
  ('minors', 0.03241859405373477),
  ('human', 0.030936539923855384)],
 [('trees', 0.3249256535151684),
  ('graph', 0.32373545741107845),
  ('minors', 0.22060806640237762),
  ('computer', 0.03819443709873786),
  ('survey', 0.022472092171356966),
  ('human', 0.010041390232247207),
  ('interface', 0.010041390232247207),
  ('response', 0.010041390232247207),
  ('time', 0.010041390232247207),
  ('user', 0.010041390232247207),
  ('system', 0.00992867112002237),
  ('eps', 0.00992867112002237)]]

In [7]:
ldaseq.print_topics(time=2)

[[('system', 0.21241936929164823),
  ('user', 0.16472754916179352),
  ('computer', 0.11755004609140896),
  ('interface', 0.11755004609140896),
  ('eps', 0.06960644191233825),
  ('response', 0.06944356260480058),
  ('time', 0.06944356260480058),
  ('survey', 0.05111543816120229),
  ('trees', 0.032396675636444675),
  ('graph', 0.032396675636444675),
  ('minors', 0.032396675636444675),
  ('human', 0.030953957171264507)],
 [('graph', 0.32400816741793875),
  ('trees', 0.32379355787054775),
  ('minors', 0.22150884150941086),
  ('computer', 0.038181772854098815),
  ('survey', 0.0224696342220385),
  ('human', 0.010037636236713735),
  ('interface', 0.010037636236713735),
  ('response', 0.010037636236713735),
  ('time', 0.010037636236713735),
  ('user', 0.010037636236713735),
  ('system', 0.009924922471198317),
  ('eps', 0.009924922471198317)]]

And that's alright: we have no reason to expect any changes! In order to see some real time variance, let's walk through how to apply LDA Sequence topic modeling to our UN general debate data.

In [59]:
import pandas as pd

un_df = pd.read_json('un-general-debates.json')

It might be interesting to see how the results of our LDA Sequence model compare to what we saw with the `post-soviet` key when running a conventional LDA model with our UN data.


To hone in on the transition from pre-Soviet to post-Soviet (and for the sake of limiting excessive run times), let's look specifically at the 660 documents in the UN dataset from the two years leading up to, and the two years following, the dissolution of the Soviet Union. 

In [10]:
a = un_df['speech_year'] < 1990

In [11]:
b = un_df['speech_year'] > 1993

In [12]:
un_df = un_df[a == False]
un_df = un_df[b == False]

<ipython-input-12-0df7f11f4650>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  un_df = un_df[b == False]


In [14]:
un_df['speech_year'].describe()

count     660.000000
mean     1991.546970
std         1.119251
min      1990.000000
25%      1991.000000
50%      1992.000000
75%      1993.000000
max      1993.000000
Name: speech_year, dtype: float64

#### Setting up our Corpus

[Corpora and Vector Spaces, Gensim style](https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html)

The collection of text documents we're looking at is contained within the 'speech_text' key of our UN dataframe:

In [50]:
un_df['speech_text']

0       ﻿It is indeed a pleasure for me and the member...
1       ﻿\nMay I begin by congratulating you. Sir, on ...
2       ﻿\nMr. President, it is a particular pleasure ...
3       ﻿\nDuring the debate at the fortieth session o...
4       ﻿I should like at the outset to express my del...
                              ...                        
3209    I should like to congratulate\nMr. Essy on his...
3210    It is with pleasure that I begin this speech b...
3211    Allow me first of all, Sir, to congratulate yo...
3212    It is a great pleasure to attend the beginning...
3213    Your election,\nSir, to the presidency of the ...
Name: speech_text, Length: 991, dtype: object

To set up our corpus with gensim, we'll define a function to iterate over each line of `speech_text`, and generate new tokenized words every time we hit white space. The `doc2bow` method available through gensim.dictionary lets us convert our documents into bag-of-words vectors. 

In [72]:
text = un_df['speech_text']

In [76]:
# 

class Corpus(object):
    def __iter__(self):
        for line in un_df['speech_text']:
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

One beneficial aspect of gensim is its ability to load one vectorized document within a corpus into memory at a time, rather than require the entire corpus be stored in RAM. This capability is particularly useful when looking at exceptionally large corpora.

In [77]:
corpus_memory_friendly = Corpus()
print(corpus_memory_friendly)

In [78]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

In [ ]:
len(vector)

#### Setting Up Our Dictionary

In [14]:
from gensim import corpora
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in un_df['speech_text'])
# remove stop words and words that appear only once
stoplist = set('i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now'.split())
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids) 
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

Dictionary(53943 unique tokens: ['(gnp)', '-', '0.15', '0.7', '1980s']...)


#### Setting up our Time Slices

Let's say we want to look at the evolution of our UN general debate topics over the two years leading up to, and the two years following, the dissolution of the Soviet Union:

- 1990-1991 = 156+162 = **318**


- 1992-1993 = 167+175 = **342**


In [20]:
time_slice = [318,342]

#### Running the Model

In [21]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [24]:
ldaseq = LdaSeqModel(corpus=corpus_memory_friendly, 
                     id2word=dictionary, 
                     time_slice=time_slice,
                     passes = 50,
                     em_max_iter=5,
                     num_topics=3)

In [25]:
ldaseq.print_topics(time=0)

[[('united', 0.012881717301908149),
  ('nations', 0.011280328877099661),
  ('international', 0.007868745327407932),
  ('must', 0.005634522173725254),
  ('economic', 0.005587701719408631),
  ('development', 0.005392324532478288),
  ('peace', 0.0051295945044163545),
  ('world', 0.005006519901349834),
  ('new', 0.0049545974768700155),
  ('countries', 0.0043914934465619756),
  ('human', 0.004117930924323075),
  ('also', 0.0040500724764122855),
  ('government', 0.0037696377673435578),
  ('general', 0.0035724675086332893),
  ('security', 0.0034434861476462953),
  ('-', 0.0032967103729869144),
  ('south', 0.0031380994523562077),
  ('community', 0.003046577646061582),
  ('states', 0.0030270886613179677),
  ('assembly', 0.0029712799674285442)],
 [('united', 0.009784826829961039),
  ('nations', 0.007910713170383901),
  ('international', 0.007469281120361953),
  ('world', 0.00655084196481777),
  ('new', 0.0063355256304786205),
  ('must', 0.005214561544132191),
  ('-', 0.0051282672652508635),
  ('

In [26]:
ldaseq.print_topics(time=1)

[[('united', 0.012912783205095419),
  ('nations', 0.011309291084115491),
  ('international', 0.007885250524048818),
  ('must', 0.005651393301183694),
  ('economic', 0.005598877067265602),
  ('development', 0.005404092244020621),
  ('peace', 0.00513766761184557),
  ('world', 0.0050202683819239174),
  ('new', 0.004964078200077779),
  ('countries', 0.004401119742382695),
  ('human', 0.004124063767902681),
  ('also', 0.004056794868209944),
  ('government', 0.0037769571575736968),
  ('general', 0.003577649447417978),
  ('security', 0.003448960358703912),
  ('-', 0.0032975861209046365),
  ('south', 0.003147722741019835),
  ('community', 0.0030524088940175067),
  ('states', 0.0030297239762284317),
  ('assembly', 0.00297587767251621)],
 [('united', 0.009807382421095415),
  ('nations', 0.007929422349966833),
  ('international', 0.007488074473874926),
  ('world', 0.006567224787098481),
  ('new', 0.006349679669450001),
  ('must', 0.005231894292532233),
  ('-', 0.005125571267160108),
  ('human', 0

## Short Text Topic Modeling

While LDA topic modeling works well when the texts in our corpus are considerably lengthy (around fifty words or more), LDA models run into some issues when applied to shorter texts. This happens because of a major assumption of LDA modeling: that each text is a *mixture of topics*. While this makes sense in the case of longer texts, shorter texts, like social media posts, often consist of only a *single topic*. 

In [30]:
# Working on it...Code's giving me trouble but that's nothing new. 

import spacy
import gsdmm

from sklearn.datasets import fetch_20newsgroups

import pickle
import matplotlib
import pandas as pd
import numpy as np
import ast

In [31]:
cats = ['talk.politics.mideast', 'comp.windows.x', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=cats)
newsgroups_train_subject = fetch_20newsgroups(subset='train', categories=cats)

data = newsgroups_train.data
data_subject = newsgroups_train_subject.data

targets = newsgroups_train.target.tolist()
target_names = newsgroups_train.target_names

## References

Alghamdi, Rubayyi and Khalid Alfalqi. 2015. "A Survey of Topic Modeling in Text Mining." *Int. J. Adv. Comput. Sci. Appl.(IJACSA)*, 6(1).

Blei, David M. and John D. Lafferty. 2006. "Dynamic Topic Models." In *Proceedings of the 23rd International Conference on Machine Learning* (pp. 113-120).

https://towardsdatascience.com/short-text-topic-modeling-70e50a57c883

https://markroxor.github.io/gensim/static/notebooks/ldaseqmodel.html